If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
# %pip install datasets transformers

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [1]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /home/ec2-user/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
# !apt install git-lfs

# with yum :
# curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.rpm.sh | sudo bash
# sudo yum install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [2]:
import transformers

print(transformers.__version__)

4.21.1


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling).

# Train a language model

In this notebook, we'll see how to train a [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling task. We will cover two types of language modeling tasks which are:

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

![Widget inference representing the causal language modeling task](https://github.com/huggingface/notebooks/blob/main/examples/images/causal_language_modeling.png?raw=1)

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.

![Widget inference representing the masked language modeling task](https://github.com/huggingface/notebooks/blob/main/examples/images/masked_language_modeling.png?raw=1)

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to train a model on it.

This notebooks assumes you have trained a tokenizer on the corpus you are using, see the [How to train a tokenizer](https://github.com/huggingface/notebooks/blob/master/examples/tokenizer_training.ipynb) notebook ([open in colab](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/tokenizer_training.ipynb)).

A script version of this notebook you can directly run on a distributed environment or on TPU is available in our [examples folder](https://github.com/huggingface/transformers/tree/master/examples).

## Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

In [3]:
from datasets import load_dataset
# datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')
datasets = load_dataset('mustapha/QuranExe', split={"train" :'train[:90%]', "validation" :'train[90%:]'})

Using custom data configuration mustapha--QuranExe-411197fcde80f3e4
Reusing dataset csv (/home/ec2-user/.cache/huggingface/datasets/mustapha___csv/mustapha--QuranExe-411197fcde80f3e4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'resource_name', 'verses_keys'],
        num_rows: 44899
    })
    validation: Dataset({
        features: ['text', 'resource_name', 'verses_keys'],
        num_rows: 4989
    })
})

You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [5]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [6]:
datasets["train"][1000]

{'text': 'قال ابن أبي حاتم: حدثنا أبي أخبرنا عبدالرحمن بن صالح أخبرنا عبدالرحمن بن محمد بن عبدالله الفزاري عن شيبان النحوي أخبرني قتادة عن عكرمة عن ابن عباس عن النبي صلى الله عليه وسلم قال أنزلت علي" إنا أرسلناك بالحق بشيرا ونذيرا" قال بشيرا بالجنة ونذيرا من النار قوله" ولا تسأل عن أصحاب الجحيم" قراءة أكثرهم ولا تسأل بضم التاء على الخبر وفي قراءة أبي بن كعب وما تسأل وفي قراءة ابن مسعود ولن تسأل عن أصحاب الجحيم نقلهما ابن جرير أي نسألك عن كفر من كفر بك كقوله "فإنما عليك البلاغ وعلينا الحساب" وكقوله تعالى "فذكر إنما أنت مذكر لست عليهم بمسيطر" الآية وكقوله تعالى "نحن أعلم بما يقولون وما أنت عليهم بجبار فذكر بالقرآن من يخاف وعيد" وأشباه ذلك من الآيات وقرأ آخرون " ولا تسأل عن أصحاب الجحيم "بفتح التاء على النهي أي لا تسأل عن حالهم كما قال عبد الرزاق: أخبرنا الثوري عن موسى بن عبيدة عن محمد بن كعب القرظي قال قال رسول الله صلى الله عليه وسلم "ليت شعري ما فعل أبواي ليت شعري ما فعل أبواي ليت شعري ما فعل أبواي؟" فنزلت "ولا تسأل عن أصحاب الجحيم فما ذكرهما حتى توفاه الله عز وجل ورواه ابن جرير عن ابن

In [7]:
datasets["train"]

Dataset({
    features: ['text', 'resource_name', 'verses_keys'],
    num_rows: 44899
})

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [8]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(datasets["train"])

As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Causal Language modeling

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or 
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`gpt2`](https://huggingface.co/gpt2) architecture for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) instead. For the tokenizer, you can replace the checkpoint by the one you trained yourself.

In [ ]:
model_checkpoint = "gpt2"
tokenizer_checkpoint = "sgugger/gpt2-like-tokenizer"

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

Downloading:   0%|          | 0.00/236 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/387k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/663k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Parameter 'function'=<function tokenize_function at 0x7fb7144729e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/10 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [ ]:
tokenized_datasets["train"][1]

{'attention_mask': [1, 1, 1, 1, 1, 1],
 'input_ids': [238, 8576, 9441, 2987, 238, 252]}

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/10 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/10 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [ ]:
tokenizer.decode(lm_datasets["train"][10]["input_ids"])

', the upper echelons of the Gallian Army exploit the concept of plausible deniability in order to send them on missions that would otherwise make Gallia lose face in the war. While at times this works to their advantage, such as a successful incursion into Imperial territory, other orders cause certain members of the 422nd great distress. One such member, Gusurg, becomes so enraged that he abandons his post and defects into the ranks of Calamity Raven, attached to the ideal of Darcsen independence proposed by their leader, Dahau. At the same time, elements within Gallian Army Command move to'

Now that the data has been cleaned, we're ready to instantiate our `Trainer`. First we create the model using the same config as our checkpoint, but initialized with random weights:

In [ ]:
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_config(config)

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

And we will needsome `TrainingArguments`:

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(
    f"{model_checkpoint}-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
)

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

We pass along all of those to the `Trainer` class:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

And we can train our model:

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 17991
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6747


Epoch,Training Loss,Validation Loss
1,6.554200,6.464636
2,6.186700,6.197421
3,6.011500,6.108301


Saving model checkpoint to gpt2-wikitext2/checkpoint-500
Configuration saved in gpt2-wikitext2/checkpoint-500/config.json
Model weights saved in gpt2-wikitext2/checkpoint-500/pytorch_model.bin
Saving model checkpoint to gpt2-wikitext2/checkpoint-1000
Configuration saved in gpt2-wikitext2/checkpoint-1000/config.json
Model weights saved in gpt2-wikitext2/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to gpt2-wikitext2/checkpoint-1500
Configuration saved in gpt2-wikitext2/checkpoint-1500/config.json
Model weights saved in gpt2-wikitext2/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to gpt2-wikitext2/checkpoint-2000
Configuration saved in gpt2-wikitext2/checkpoint-2000/config.json
Model weights saved in gpt2-wikitext2/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1934
  Batch size = 8
Saving model checkpoint to gpt2-wikitext2/checkpoint-2500
Configuration saved in gpt2-wikitext2/checkpoint-2500/config.json
Model weights saved in gp

TrainOutput(global_step=6747, training_loss=6.38550191867682, metrics={'train_runtime': 2281.5205, 'train_samples_per_second': 23.657, 'train_steps_per_second': 2.957, 'total_flos': 3525678710784000.0, 'train_loss': 6.38550191867682, 'epoch': 3.0})

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1934
  Batch size = 8


Perplexity: 449.57


The perplexity is still quite high since for this demo we trained on a small dataset for a small number of epochs. For a real LM training, you  would need a larger dataset and more epochs.

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

KeyboardInterrupt: ignored

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sgugger/my-awesome-model")
```

## Masked language modeling

In [10]:
from transformers import AutoTokenizer

In [11]:
block_size = 128
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [12]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

For masked language modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by `[MASK]`) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens). If you use a tokenizer you trained yourself, make sure the `[MASK]` token is among the special tokens you passed during training!

We will use the [`bert-base-cased`](https://huggingface.co/bert-based-cased) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=masked-lm) instead. For the tokenizer, replace the checkpoint by the one you trained.

In [13]:
# model_checkpoint = "bert-base-cased"
# tokenizer_checkpoint = "sgugger/bert-like-tokenizer"
model_checkpoint = "aubmindlab/bert-base-arabertv02"
tokenizer_checkpoint = "aubmindlab/bert-base-arabertv02"

We can apply the same tokenization function as before, we just need to update our tokenizer to use the checkpoint we just picked:

In [14]:
# datasets["train"] = datasets["train"]["text"]

In [15]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text", 'resource_name', 'verses_keys'])

Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/mustapha___csv/mustapha--QuranExe-411197fcde80f3e4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-3e0c7c859186c02d.arrow


Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/mustapha___csv/mustapha--QuranExe-411197fcde80f3e4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-ddf6d873352e8ebb.arrow


Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/mustapha___csv/mustapha--QuranExe-411197fcde80f3e4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-9489e70efbafafc4.arrow


Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/mustapha___csv/mustapha--QuranExe-411197fcde80f3e4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-1de0103c968cf328.arrow


Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/mustapha___csv/mustapha--QuranExe-411197fcde80f3e4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-7b165a1519850c63.arrow


Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/mustapha___csv/mustapha--QuranExe-411197fcde80f3e4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-f95c01b215b1ea6b.arrow


Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/mustapha___csv/mustapha--QuranExe-411197fcde80f3e4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-78cfe512a5d3bb0c.arrow


Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/mustapha___csv/mustapha--QuranExe-411197fcde80f3e4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-6e455c8155824ec5.arrow


And like before, we group texts together and chunk them in samples of length `block_size`. You can skip that step if your dataset is composed of individual sentences.

In [16]:
tokenized_datasets["train"]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 44899
})

In [17]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=512,
    num_proc=4,
)

Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/mustapha___csv/mustapha--QuranExe-411197fcde80f3e4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-96453f15b3eab2ce.arrow


Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/mustapha___csv/mustapha--QuranExe-411197fcde80f3e4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-81cf4c2134fffc46.arrow


Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/mustapha___csv/mustapha--QuranExe-411197fcde80f3e4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-9220927a56834991.arrow


Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/mustapha___csv/mustapha--QuranExe-411197fcde80f3e4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-ba633daea53d19b8.arrow


Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/mustapha___csv/mustapha--QuranExe-411197fcde80f3e4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-9f8d0fab40c72912.arrow


Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/mustapha___csv/mustapha--QuranExe-411197fcde80f3e4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-5f89247c9b9f20f5.arrow


Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/mustapha___csv/mustapha--QuranExe-411197fcde80f3e4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-d2ac9fa9fdb73cb0.arrow


Loading cached processed dataset at /home/ec2-user/.cache/huggingface/datasets/mustapha___csv/mustapha--QuranExe-411197fcde80f3e4/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-b5cab204b4276c93.arrow


The rest is very similar to what we had, with two exceptions. First we use a model suitable for masked LM:

In [18]:
from transformers import AutoConfig, AutoModelForMaskedLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_config(config)

We redefine our `TrainingArguments`:

In [19]:
from transformers import Trainer, TrainingArguments

In [20]:
training_args = TrainingArguments(
    "QuranExe-bert",
    per_device_train_batch_size = 16,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    save_total_limit=2,
    # push_to_hub_model_id=f"QuranExe-bert",
    hub_model_id="mustapha/QuranExe-bert",
    report_to="wandb"
)

Like before, the last two arguments are to setup everything so we can push the model to the [Hub](https://huggingface.co/models) at the end of training. Remove the two of them if you didn't follow the installation steps at the top of the notebook, otherwise you can change the value of `push_to_hub_model_id` to something you would prefer.

Finally, we use a special `data_collator`. The `data_collator` is a function that is responsible of taking the samples and batching them in tensors. In the previous example, we had nothing special to do, so we just used the default for this argument. Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the `data_collator`, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a `DataCollatorForLanguageModeling`. We can adjust the probability of the masking:

In [21]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Then we just have to pass everything to `Trainer` and begin training:

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

/home/ec2-user/islamic_search/training/QuranExe-bert is already a clone of https://huggingface.co/mustapha/QuranExe-bert. Make sure you pull the latest changes with `repo.git_pull()`.


In [24]:
# trainer.train(resume_from_checkpoint="QuranExe-bert/checkpoint-6500")
trainer.train(resume_from_checkpoint=True)
# trainer.train()

Loading model from QuranExe-bert/checkpoint-26000.
/home/ec2-user/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 141460
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 26526
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 2
  Continuing training from global step 26000
  Will skip the first 2 epochs then the first 8316 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already see

  0%|          | 0/8316 [00:00<?, ?it/s]

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mustapha. Use `wandb login --relogin` to force relogin
wandb: ERROR Error while calling W&B API: Error 1062: Duplicate entry '821533-1cj40uz2' for key 'PRIMARY' (<Response [409]>)


Epoch,Training Loss,Validation Loss
3,6.380500,6.327925


Saving model checkpoint to QuranExe-bert/checkpoint-26500
Configuration saved in QuranExe-bert/checkpoint-26500/config.json
Model weights saved in QuranExe-bert/checkpoint-26500/pytorch_model.bin
Several commits (2) will be pushed upstream.
Deleting older checkpoint [QuranExe-bert/checkpoint-25500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 7376
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=26526, training_loss=0.12649399283460244, metrics={'train_runtime': 2910.761, 'train_samples_per_second': 145.797, 'train_steps_per_second': 9.113, 'total_flos': 2.793773235358925e+16, 'train_loss': 0.12649399283460244, 'epoch': 3.0})

Like before, we can evaluate our model on the validation set. The perplexity is much lower than for the CLM objective because for the MLM objective, we only have to make predictions for the masked tokens (which represent 15% of the total here) while having access to the rest of the tokens. It's thus an easier task for the model.

In [26]:
import math 
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 7376
  Batch size = 8


Perplexity: 558.33


The perplexity is still quite high since for this demo we trained on a small dataset for a small number of epochs. For a real LM training, you  would need a larger dataset and more epochs.

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("sgugger/my-awesome-model")
```